In [94]:
# Imports

import sys
sys.path.append('../src/cosucra/2_transformation')
import transformation
import pandas as pd


In [266]:
# Reload exploration file
import importlib
importlib.reload(transformation)


<module 'transformation' from '/Users/baudouinm/Documents/Dataroots/cosucra/notebooks/../src/cosucra/2_transformation/transformation.py'>

Import data from exploration

In [96]:
import pickle

# Loading the data from a file
with open('data.pkl', 'rb') as f:
    process_data = pickle.load(f)


In [97]:
processA = process_data["A"]
processC = process_data["C"]
processE3 = process_data["E3"]
processG2 = process_data["G2"]
processH3 = process_data["H3"]
processP2 = process_data["P2"]
processPP2 = process_data["PP2"]
processT2 = process_data["T2"]
processU = process_data["U"]


Renaming and aggregation Timestamp column by 30 minutes.

In [ ]:
process = ["A", "C", "E3", "G2", "H3", "P2", "PP2", "T2", "U"]

# Dictionary to hold dataframes for each process (assuming they're pre-loaded)
# For example, if processA, processC, etc., are already loaded into variables, store them here
process_data = {
    'A': processA,  # Assuming processA is a preloaded DataFrame
    'C': processC,
    'E3': processE3,
    'G2': processG2,
    'H3': processH3,
    'P2': processP2,
    'PP2': processPP2,
    'T2': processT2,
    'U': processU
}

all_results = transformation.apply_transformation(process, process_data)

print(all_results)


Implementation of the subprocesses lags

In [283]:
tag_map = {
    "A": 0,
    "T2_1": 120,
    "T2_2": 420,
    "T2_3": 480,
    "E3": 480,
    "P2_1": 510,
    "P2_2": 540,
    "PP2": 540,
    "G2_1": 570,
    "G2_2": 600
}

tag_map_shift = {
    "U": 0,
    "C": 0,
    "A": 0,
    "T2_1": 4,
    "T2_2": 14,
    "T2_3": 16,
    "E3": 16,
    "P2_1": 17,
    "P2_2": 18,
    "PP2": 18,
    "G2_1": 19,
    "G2_2": 20
}


In [284]:
processUShifted = transformation.shift_non_timestamp_columns(all_results["U"], tag_map_shift["U"])
processCShifted = transformation.shift_non_timestamp_columns(all_results["C"], tag_map_shift["C"])
processAShifted = transformation.shift_non_timestamp_columns(all_results["A"], tag_map_shift["A"])
shifts = {'TT_T2_14_1': tag_map_shift["T2_1"], 'T2_Drakkar_Flag_Prod': tag_map_shift["T2_2"], 'LT_T2_24': tag_map_shift["T2_3"]}
boundaries = ['TT_T2_14_1', 'T2_Drakkar_Flag_Prod', 'LT_T2_24']
processT2Shifted = transformation.shift_custom_columns_generic(all_results["T2"], shifts, boundaries)
processE3Shifted = transformation.shift_non_timestamp_columns(all_results["E3"], tag_map_shift["E3"])
shifts_P2 = {'PT_P2_01_2': tag_map_shift["P2_1"], 'ECH_PH_PP2_1': tag_map_shift["P2_2"]}
boundaries_P2 = ['PT_P2_01_2', 'ECH_PH_PP2_1']
processP2Shifted = transformation.shift_custom_columns_generic(all_results["P2"], shifts_P2, boundaries_P2)
processPP2Shifted = transformation.shift_non_timestamp_columns(all_results["PP2"], tag_map_shift["PP2"])
shifts_G = {'H3_Atomiseur_Flag_Prod': tag_map_shift["G2_1"], 'PT_G2_05': tag_map_shift["G2_2"]}
boundaries_G = ['H3_Atomiseur_Flag_Prod', 'PT_G2_05']
processGShifted = transformation.shift_custom_columns_generic(all_results["G2"], shifts_G, boundaries_G)


In [285]:
dataset_array = [processUShifted,processCShifted,processAShifted, processT2Shifted,processE3Shifted, processP2Shifted, processPP2Shifted, processGShifted ]


Merge of all subprocesses

In [286]:
merged_df = transformation.merge_datasets_on_timestamp(dataset_array)
print(merged_df)


                TimeStamp  FT_U09_1  FT_U09_2  FT_U10_1  FT_U09_17_1  \
0     2023-01-17 00:00:00     49.80      0.00     19.49        53.92   
1     2023-01-17 00:30:00     49.72      0.00     20.94        57.36   
2     2023-01-17 01:00:00     49.68      0.00     21.60        49.72   
3     2023-01-17 01:30:00     49.73      0.00     20.70        58.72   
4     2023-01-17 02:00:00     49.66      0.00     20.26        49.40   
...                   ...       ...       ...       ...          ...   
11659 2023-09-16 21:30:00      0.00     72.66      0.15        60.70   
11660 2023-09-16 22:00:00      0.00     72.49      0.20        60.82   
11661 2023-09-16 22:30:00      0.00     71.24      0.21        60.65   
11662 2023-09-16 23:00:00      0.00     68.51      0.58        60.54   
11663 2023-09-16 23:30:00      0.00      1.47      0.72        60.56   

       AT_U09_17_1  FT_U09_3  JT_U06_02  TT_U07_07_04  TT_U01_0801_1  ...  \
0             7.74    103.82    1468.40         95.34     

Null values computation for the merge dataframe

In [276]:
null_values = merged_df.isnull().sum()
print(null_values)


TimeStamp           0
FT_U09_1           26
FT_U09_2           26
FT_U10_1           26
FT_U09_17_1        26
                ...  
DT_G2_1001         48
PT_G2_1002_1       47
FT_G2_1001_1       47
FT_G2_1002         47
TT_G2_1002      10794
Length: 318, dtype: int64


In [287]:
merged_df


,TimeStamp,FT_U09_1,FT_U09_2,FT_U10_1,FT_U09_17_1,AT_U09_17_1,FT_U09_3,JT_U06_02,TT_U07_07_04,TT_U01_0801_1,...,LT_G2_10,TT_G2_09,LT_G2_09,PT_G2_10_2,TT_G2_1001,DT_G2_1001,PT_G2_1002_1,FT_G2_1001_1,FT_G2_1002,TT_G2_1002
0,2023-01-17 00:00:00,49.80,0.00,19.49,53.92,7.74,103.82,1468.40,95.34,64.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-17 00:30:00,49.72,0.00,20.94,57.36,7.71,109.56,1468.50,95.88,64.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-17 01:00:00,49.68,0.00,21.60,49.72,7.71,98.72,1467.43,95.96,64.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-17 01:30:00,49.73,0.00,20.70,58.72,7.73,91.45,1466.95,95.47,64.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-17 02:00:00,49.66,0.00,20.26,49.40,7.71,91.67,1467.95,95.56,64.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11659,2023-09-16 21:30:00,0.00,72.66,0.15,60.70,6.94,104.73,1467.78,98.59,64.76,...,1164.52,82.11,824.71,4.77,83.32,1025.82,6.50,7695.63,2267.97,87.73
11660,2023-09-16 22:00:00,0.00,72.49,0.20,60.82,6.96,98.87,1468.50,100.74,64.82,...,1130.76,82.00,820.18,5.63,83.30,1025.87,6.50,7689.69,2267.19,87.64
11661,2023-09-16 22:30:00,0.00,71.24,0.21,60.65,6.96,96.34,1466.85,99.10,64.76,...,1104.77,81.93,812.86,4.56,83.19,1026.14,6.50,7711.56,2251.56,87.53
11662,2023-09-16 23:00:00,0.00,68.51,0.58,60.54,6.96,100.18,1467.67,99.70,64.65,...,1056.66,82.07,811.17,5.38,83.18,1025.93,6.49,7685.31,2248.44,87.53


Create visual presentation of the lag values

In [141]:
window_df = transformation.window_by_column_name(merged_df, "PT_E3_03_11", num_cols=4)
transformation.save_df_as_png(window_df, filename="./plots/ProcessC/preprocessedProcessA.png", num_rows=20)


In [136]:
window_df


,ECH_PURNIR_POIS,ECH_TX_COQUES,TT_T2_14_1,TT_T2_14_1
0,22.32,3.28,NaN,NaN
1,22.49,3.46,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,56.81,NaN
...,...,...,...,...
11659,NaN,NaN,62.24,61.40
11660,NaN,NaN,73.43,61.90
11661,25.21,4.05,60.57,55.39
11662,NaN,NaN,61.11,55.23


PCA computation

In [ ]:
pca_df, pca_model = transformation.perform_pca(merged_df, n_components=317)

# Display the result of PCA
print(pca_df)

# Explained variance ratio for each component
print("Explained variance ratio:", pca_model.explained_variance_ratio_)


In [269]:
# Get the top 5 values and their indices
top_values, top_indices = transformation.get_top_values_indices(pca_model.explained_variance_ratio_, top_n=20)

# Get the column names based on the top indices
column_names = transformation.get_column_names_from_indices(merged_df, top_indices)

print("Top Values:", top_values)
print("Indices:", top_indices)
print("Column Names:", column_names)


Top Values: [0.40464962 0.04682839 0.03774965 0.02554809 0.0225885  0.01628274
 0.01484574 0.01371398 0.01222883 0.01084286 0.01078945 0.01008454
 0.00934037 0.00871793 0.00825253 0.00788477 0.00749449 0.00711665
 0.00694084 0.00684187]
Indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Column Names: ['TimeStamp', 'FT_U09_1', 'FT_U09_2', 'FT_U10_1', 'FT_U09_17_1', 'AT_U09_17_1', 'FT_U09_3', 'JT_U06_02', 'TT_U07_07_04', 'TT_U01_0801_1', 'PT_U01_3', 'WT_C11_CPT', 'LT_C_29_03', 'ECH_MSNIR_POIS', 'ECH_PURNIR_POIS', 'ECH_TX_COQUES', 'TT_T2_14_1', 'TT_T2_14_1', 'TT_T2_14_2', 'TT_T2_14_2']


In [ ]:
transformation.plot_correlation_matrix(transformation.get_columns_data(merged_df,['TimeStamp', 'FT_U09_1', 'FT_U09_2', 'FT_U10_1', 'FT_U09_17_1', 'AT_U09_17_1', 'FT_U09_3', 'JT_U06_02', 'TT_U07_07_04', 'TT_U01_0801_1', 'PT_U01_3', 'WT_C11_CPT', 'LT_C_29_03', 'ECH_MSNIR_POIS', 'ECH_PURNIR_POIS', 'ECH_TX_COQUES', 'TT_T2_14_1', 'TT_T2_14_1', 'TT_T2_14_2', 'TT_T2_14_2']))


Correlation computation

In [252]:
spearman_corr = merged_df.corr()
# Replace 'X' and 'Y' with the actual column names
correlations_with_X = transformation.remove_duplicate_columns(spearman_corr['FT_T2_Pois_Entier'].drop('FT_T2_Pois_Entier'))  # Drop self-correlation
correlations_with_Y = transformation.remove_duplicate_columns(spearman_corr['FT_G2_0402_MS'].drop('FT_G2_0402_MS'))


In [253]:
correlations_with_X


,FT_T2_Pois_Entier,FT_T2_Pois_Entier
TimeStamp,-0.065981,-0.066237
FT_U09_1,0.244762,0.235802
FT_U09_2,0.319533,0.309688
FT_U10_1,0.131359,0.126675
FT_U09_17_1,0.319635,0.313691
...,...,...
DT_G2_1001,0.619906,0.651509
PT_G2_1002_1,0.764531,0.797780
FT_G2_1001_1,0.587279,0.613903
FT_G2_1002,0.083452,0.088820


In [254]:

correlations_with_X = correlations_with_X.loc[:, ~correlations_with_X.columns.duplicated()]
correlations_with_Y = correlations_with_Y.loc[:, ~correlations_with_Y.columns.duplicated()]



In [255]:
corr_Target1 = correlations_with_X.sort_values(ascending=False, by="FT_T2_Pois_Entier").head(10)
corr_Target2 = correlations_with_Y.sort_values(ascending=False, by="FT_G2_0402_MS").head(10)


In [256]:
corr_Target1 = corr_Target1.reset_index().rename(columns={'index': 'Features'})
corr_Target2 = corr_Target2.reset_index().rename(columns={'index': 'Features'})


In [257]:

transformation.save_df_as_png(corr_Target1, filename="./plots/corr/corrFT_T2_Pois_Entier.png", num_rows=20)
transformation.save_df_as_png(corr_Target2, filename="./plots/corr/corrFT_G2_0402_MS.png", num_rows=20)


In [258]:
corr_Target2


,Features,FT_G2_0402_MS
0,DT_G2_0402_MS,0.951050
1,PT_G2_0402,0.931508
2,DT_G2_0402_MS,0.925991
3,PT_G2_0402,0.903521
4,TT_G2_05,0.900317
5,H3_Atomiseur_Flag_Prod,0.898458
6,SC_T2_2402,0.887490
7,PT_G2_0601,0.876896
8,FT_P2_0101,0.876735
9,H3_Atomiseur_Flag_Prod,0.866411
